In [5]:

import numpy as np


# 加载音素标签和MFCC特征的 .npy 文件
phoneme_labels_file = 'E:/cv_data/cmu11785_data/HW3P2_data/11-785-s24-hw3p2/train-clean-100/transcript/19-198-0000.npy'  # 替换为实际音素标签文件路径
mfcc_features_file = 'E:/cv_data/cmu11785_data/HW3P2_data/11-785-s24-hw3p2/train-clean-100/mfcc/19-198-0000.npy'  # 替换为实际MFCC特征文件路径

# 加载文件
phoneme_labels = np.load(phoneme_labels_file)
mfcc_features = np.load(mfcc_features_file)

# 打印音素标签和对应的MFCC数据的帧数
print("音素标签内容：")
print(phoneme_labels)
print("\nMFCC数据的形状（样本数，帧数，特征维度）：")
print(mfcc_features.shape)

# 获取MFCC数据的帧数
mfcc_frame_count = mfcc_features.shape[0]
print(f"\nMFCC数据的帧数：{mfcc_frame_count}")

音素标签内容：
['[SOS]' '[SIL]' 'N' 'AO' 'R' 'TH' 'AH' 'N' 'JH' 'ER' 'AE' 'B' 'IY'
 '[SIL]' '[EOS]']

MFCC数据的形状（样本数，帧数，特征维度）：
(192, 27)

MFCC数据的帧数：192


None
